In [ ]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "ndq.ax"
data = yf.download(ticker, period="1y", interval="1h")


In [ ]:
# Importing through a CSV that has more data which was downloaded externally.
data_csv = pd.read_csv('../Data/AUDCAD_H1(22007-09-26 - 2024-02-02).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
data_csv.index = pd.to_datetime(data_csv.index)


In [ ]:
df = data_csv
df_slice = df[:10000]

In [ ]:
# csv slice by date
start_date = '2010-01-01'
end_date = '2024-01-01'
df = data_csv.loc[start_date:end_date]

# Write out strategy here to figure out the logic

In [ ]:
class Strat(Strategy):
    def init(self):
        pass

    def next(self):
        pass

bt = Backtest(df_slice, Strat, cash=10_000)
bt.run()
bt.plot()

In [ ]:
class momo(Strategy):    
    def init(self):
        self.atr = self.I(ta.atr, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), length=14)
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), 50)        
        self.pullback_count = 0
        self.above_ema = False

    def next(self):
        price = self.data.Close[-1] 
        atr = self.atr[-1]
        ema50 = self.ema50[-1]
        
        if price > ema50:
            if not self.above_ema:
                self.above_ema = True
                self.pullback_count = 0
            else:
                if self.pullback_count >= 2:
                    if len(self.data.Close) >= 2 and is_engulfing(True, {'Open': self.data.Open[-1], 'Close': self.data.Close[-2]}) and price > ema50:
                        self.buy()
                    elif price < self.data.Close[-2]:
                        self.pullback_count += 1
                    else:
                        self.above_ema = False
                        self.pullback_count = 0

In [ ]:
bt = Backtest(df, momo, cash=100000)
stats = bt.run()
print(stats)
# bt.plot()

In [ ]:
print(stats._trades['PnL'] < 0)


In [ ]:
print(stats._strategy)

In [ ]:

# plot own equity curve based on backtesting data for data more than 10K
equity_curve = stats._equity_curve['Equity']

plt.figure(figsize=(10, 6))
plt.plot(equity_curve, label='Equity Curve', lw=1)  # lw is line width
plt.title('Equity Curve')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(False)
plt.show()


In [ ]:

equity_curve = stats._equity_curve['Equity'].reset_index()

fig = px.line(equity_curve, x='index', y='Equity', labels={'index': 'Time'}, title='Strategy Performance')
fig.update_layout(height=600, xaxis_title='Time', yaxis_title='Equity', legend_title='Legend')
fig.show()
